## In-context Table Self-query with Intelligent Document Processing
---

<div class="alert alert-block alert-info"> 
    <b>NOTE:</b> You will need to use a Jupyter Kernel with Python 3.9 or above to use this notebook. If you are in Amazon SageMaker Studio, you can use the "Data Science 3.0" image.
</div>

<div class="alert alert-block alert-warning"> 
    <b>NOTE:</b> You will need 3rd party model access to Anthropic Claude V1 model, and Amazon Titan Embedding G1 Text model to be able to run this notebook. Verify if you have access to the models by going to <a href="https://console.aws.amazon.com/bedrock" target="_blank">Amazon Bedrock console</a> > left menu "Model access". The "Access status" for Anthropic Claude and Amazon Titan Embedding G1 Text must be in "<span style="color:green;">Access granted</span>" status. If you do not have access, then click "Edit" button on the top right > select the model checkboxes > click "Save changes" button at the bottom. You should have access to the model within a few moments.
</div>

In this notebook we will walk through how to perform _"self querying"_ with table data wth tables present in documents. First we will be extracting the tables from a document using Amazon Textract using `AnalyzeDocument` API, generating the table data and then store the table data into a Vector DB in a very specific way, and then performing self-querying on the table data with a Anthropic Claude model via Amazon Bedrock and get precise answers from the model. 

In [1]:
!pip install -U boto3 langchain chromadb lark
!pip install amazon-textract-textractor amazon-textract-prettyprinter pypdf Pillow
!pip install sqlalchemy --upgrade

  Using cached SQLAlchemy-2.0.28-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (9.6 kB)
Using cached SQLAlchemy-2.0.28-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (3.1 MB)
  Attempting uninstall: sqlalchemy
    Found existing installation: SQLAlchemy 1.4.50
    Uninstalling SQLAlchemy-1.4.50:
      Successfully uninstalled SQLAlchemy-1.4.50
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
jupyter-scheduler 2.4.0 requires sqlalchemy~=1.0, but you have sqlalchemy 2.0.28 which is incompatible.


In [2]:
import json
import os
import sys
import sagemaker
import boto3

role = sagemaker.get_execution_role()
data_bucket = sagemaker.Session().default_bucket()
bedrock = boto3.client('bedrock-runtime', region_name='us-west-2')
br = boto3.client('bedrock')
s3 = boto3.client("s3")
print(f"SageMaker bucket is {data_bucket}, and SageMaker Execution Role is {role}")

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/sagemaker-user/.config/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/sagemaker-user/.config/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/sagemaker-user/.config/sagemaker/config.yaml
SageMaker bucket is sagemaker-us-east-2-364001846372, and SageMaker Execution Role is arn:aws:iam::364001846372:role/aws-idp-workshop-SageMakerExecutionRole-onv4uhuHNlPZ


## Extract table data from the document using Amazon Textract
---

The sample document is in `/samples` directory. For this workshop, we will be using a sample bank statement document (`bank_statement.jpg`) that contains tables data. We will use the `amazon-textract-textractor` library to perform the API call to `AnalyzeDocument` with `TABLE` feature and also read the table data with the Textract response parser. Once the tables are extracted we will parse out 

In [3]:
from textractcaller.t_call import call_textract, Textract_Features
from textractprettyprinter.t_pretty_print import Pretty_Print_Table_Format, Textract_Pretty_Print, get_string

textract_json = call_textract(input_document="./samples/bank_statement.jpg", features=[Textract_Features.TABLES])

print(get_string(textract_json=textract_json,
               table_format=Pretty_Print_Table_Format.tsv,
               output_type=[Textract_Pretty_Print.TABLES]))

Date      	Description             	Deposits ($)	Withdrawals ($)	Amount ($)
10/06/2021	Opening Balance         	            	               	4,250.00
10/07/2021	Cash Deposit            	1,250.00    	               	5,500.00
10/10/2021	Utility Charge          	            	245.35         	5,254.65
10/15/2021	Cheque Deposit          	750.00      	               	6,004.65
10/17/2021	Bill Payment - lululemon	            	350.25         	5,654.40
10/22/2021	Online Shopping         	            	380.00         	5,274.40
10/25/2021	Online Transfer         	            	450.00         	4,824.40
10/24/2021	Salary Deposit          	300.00      	               	5,124.40
10/26/2021	ATM Deposit             	150.00      	               	5,274.40
10/27/2021	Bill Payment -GM Mart   	            	360.40         	4,914.00
10/28/2021	Water Bill Payment      	            	150.00         	4,764.00
11/01/2021	Online Transfer         	480.00      	               	5,244.00
11/04/2021	Electric Bill Payment   	

We notice that Textract has extracted two distinct tables from the document. In this walkthrough we will get the first table and perform Self-query on it using Langchain. There are two tables in this page, let's do Q&A on the first table. Note that we are going to use LangChain's `SelfQueryRetriever` which is helpful with Q&A with tables. As of this writing, FAISS is not supported for self-querying with LangChain, so we will use ChromaDB. For more information refer to the [Self-query LangChain documentation](https://python.langchain.com/docs/modules/data_connection/retrievers/self_query/).

## Transform the extracted table data
---

Self query needs the table data to be formatted in a very specific way using LangChain's `Document` model. For example, here is what the structure looks like

Table
<table>
    <tr>
        <th>year</th>
        <th>director</th>
        <th>rating</th>
        <th>movie</th>
        <th>actor</th>
    </tr>
    <tr>
        <td>2010</td>
        <td>Christopher Nolan</td>
        <td>8.2</td>
        <td>Inception</td>
        <td>Leo DiCaprio</td>
    </tr>
    <tr>
        <td>2006</td>
        <td>Satoshi Kon</td>
        <td>8.6</td>
        <td>Paprika</td>
        <td>Megumi Hayashibara</td>
    </tr>
</table>

```python
docs = [
    Document(
        page_content="2010, Christopher Nolan, 8.2, Inception, Leo DiCaprio",
        metadata={"year": 2010, 
                  "director": "Christopher Nolan", 
                  "rating": 8.2, 
                  "movie": "Inception", 
                  "actor": "Leo DiCaprio"},
    ),
    Document(
        page_content="2006, Satoshi Kon, 8.6, Paprika, Megumi Hayashibara",
        metadata={"year": 2006, 
                  "director": "Satoshi Kon", 
                  "rating": 8.6,
                  "movie": "Paprika",
                  "actor": "Megumi Hayashibara"},
    )
    ...
]
```

Above, the table data rows represented by CSV string resides in the `page_content` key in the Document schema. The `metadata` section contains key-value pairs which are table header to cell value. The table may look something like below.

We will transform the first table in the document using the same schema. We will do this by accessing the individual row/col data available in the Textract output using Textract response parser utility tool. Note that our table contains numbers and as such for self-query to work we need to convert numbers into int or float type appropriately as well.

In [4]:
import csv
from io import StringIO
from trp import Document as TDoc
from langchain.schema import Document

doc = TDoc(textract_json)
rows = []

def detect_type(s):
    if type(s) == 'NoneType': 
        return s
    elif not isinstance(s, str):
        s = str(s)
    s = s.replace(',', '')
    try:
        return int(s)
    except ValueError:
        try:
            return float(s)
        except ValueError:
            return s

# Extract the first table data
for page in doc.pages:
    if page.tables:
        for row in page.tables[0].rows:
            cells = [detect_type(cell.text.strip()) for cell in row.cells]
            rows.append(cells)

headers = rows[0]
full_table = []

for row in rows[1:]:
    output = StringIO()
    csv_writer = csv.writer(output)
    csv_writer.writerow(row)
    csv_string = output.getvalue()
    row_meta = {headers[i]: detect_type(cell) for i, cell in enumerate(row)}
    full_table.append(Document(page_content=csv_string.strip(), metadata=row_meta))

full_table


[Document(page_content='10/06/2021,Opening Balance,,,4250.0', metadata={'Date': '10/06/2021', 'Description': 'Opening Balance', 'Deposits ($)': '', 'Withdrawals ($)': '', 'Amount ($)': 4250.0}),
 Document(page_content='10/07/2021,Cash Deposit,1250.0,,5500.0', metadata={'Date': '10/07/2021', 'Description': 'Cash Deposit', 'Deposits ($)': 1250.0, 'Withdrawals ($)': '', 'Amount ($)': 5500.0}),
 Document(page_content='10/10/2021,Utility Charge,,245.35,5254.65', metadata={'Date': '10/10/2021', 'Description': 'Utility Charge', 'Deposits ($)': '', 'Withdrawals ($)': 245.35, 'Amount ($)': 5254.65}),
 Document(page_content='10/15/2021,Cheque Deposit,750.0,,6004.65', metadata={'Date': '10/15/2021', 'Description': 'Cheque Deposit', 'Deposits ($)': 750.0, 'Withdrawals ($)': '', 'Amount ($)': 6004.65}),
 Document(page_content='10/17/2021,Bill Payment - lululemon,,350.25,5654.4', metadata={'Date': '10/17/2021', 'Description': 'Bill Payment - lululemon', 'Deposits ($)': '', 'Withdrawals ($)': 350.25,


## Store the table in Vector DB
---

We will now store this into our vector database by first generating embeddings. 

In [5]:
from langchain.embeddings import BedrockEmbeddings
from langchain.vectorstores import Chroma

# Ensure that you have enabled amazon.titan-embed-text-v1 model in Amazon Bedrock console
embeddings = BedrockEmbeddings(client=bedrock,model_id="amazon.titan-embed-text-v1")
vector_db = Chroma.from_documents(documents=full_table,embedding=embeddings)

In [6]:
# vector_db.delete_collection()

## Self Query Retriever with Amazon Bedrock and Anthropic Claude
---

We will now create a self-query retriever, much like the retriever we used in the _"In-context QA"_ notebook. However this time we will use some additional information to create the retriever in addition to the vector database. We created a special structure using the table data in the previous code cell (`full_table`), we will also need to define the table definition using LangChain's `AttributeInfo` model. This will help the LLM understand what each of the column/header actually means.

In [9]:
from langchain_community.llms import Bedrock
from langchain.retrievers.self_query.base import SelfQueryRetriever
from langchain.chains.query_constructor.base import AttributeInfo

metadata_field_info = [
    AttributeInfo(
        name="Date",
        description="Date of the bank transaction",
        type="string",
    ),
    AttributeInfo(
        name="Description",
        description="Description of the bank transaction",
        type="string",
    ),
    AttributeInfo(
        name="Deposits ($)",
        description="The dollar amount deposited into the bank account",
        type="integer",
    ),
    AttributeInfo(
        name="Withdrawals ($)",
        description="The dollar amount withdrawn from the bank account",
        type="integer",
    ),
    AttributeInfo(
        name="Amount ($)",
        description="The total dollar amount balance in the bank account",
        type="integer",
    )
]
document_content_description = "A transaction in a bank statement"

bedrock_llm = Bedrock(client=bedrock, model_id="anthropic.claude-instant-v1")
retriever = SelfQueryRetriever.from_llm(
    bedrock_llm, vector_db, document_content_description, metadata_field_info, verbose=True
)

Let's run a query where the withdrawal amount was greater than 300.

In [10]:
response=retriever.get_relevant_documents("What are the transactions with withdrawals greater than 300")
for r in response:
    print(r.page_content)

10/27/2021,Bill Payment -GM Mart,,360.4,4914.0
10/17/2021,Bill Payment - lululemon,,350.25,5654.4
10/25/2021,Online Transfer,,450.0,4824.4
10/22/2021,Online Shopping,,380.0,5274.4


Let's run a query where the transaction amount was greater than 5000.

In [11]:
response=retriever.get_relevant_documents("What are the transactions with deposits greater than 400")
for r in response:
    print(r.page_content)

11/01/2021,Online Transfer,480.0,,5244.0
10/15/2021,Cheque Deposit,750.0,,6004.65
10/07/2021,Cash Deposit,1250.0,,5500.0
